In [ ]:
# Набросок каркаса уплотнения контекста
# Установка необходимых библиотек
# !pip install torch transformers datasets


# Импорт необходимых модулей
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch import nn, optim
from datasets import load_dataset


# Загрузка модели GPT-2 и токенизатора
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


# Перевод модели в режим оценки
model.eval()



# Определение модуля оптимизации контекста
class ContextOptimizer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ContextOptimizer, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, input_size)
        self.relu = nn.ReLU()


    def forward(self, context):
        x = self.relu(self.fc1(context))
        x = self.fc2(x)
        return x



# Параметры оптимизатора
input_size = 768  # Размер скрытого состояния модели GPT-2
hidden_size = 256
optimizer = ContextOptimizer(input_size, hidden_size)


# Загрузка датасета "wikitext"
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")



# Функция для подготовки данных: извлечение контекста и текущего фрагмента
def prepare_data(example, max_length=50):
    text = example["text"].strip()
    if len(text) == 0 or "." not in text:
        return None, None


    # Разделение текста на предысторию и текущий фрагмент
    split_point = text.find(".") + 1
    if split_point >= len(text):
        return None, None


    context = text[:split_point].strip()  # Предыстория до первой точки
    current_fragment = text[split_point : split_point + max_length].strip()  # Текущий фрагмент

    if len(context) == 0 or len(current_fragment) == 0:
        return None, None

    return context, current_fragment



# Функция для сжатия контекста
def compress_context(context, optimizer):
    # Получение эмбеддингов контекста из токенов
    inputs = tokenizer(context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        context_embedding = outputs.hidden_states[-1].mean(dim=1)  # Среднее по последнему слою


    # Сжатие контекста через оптимизатор
    compressed_context = optimizer(context_embedding)
    return compressed_context



# Функция предсказания на основе сжатого контекста
def predict_with_compressed_context(compressed_context, current_fragment):
    inputs = tokenizer(current_fragment, return_tensors="pt")
    input_ids = inputs["input_ids"]


    # Вставка сжатого контекста в модель
    with torch.no_grad():

        outputs = model(input_ids=input_ids)
        logits = outputs.logits


    # Получение предсказаний
    predicted_index = torch.argmax(logits[0, -1, :]).item()
    predicted_token = tokenizer.decode([predicted_index])
    return predicted_token



# Функция потерь и обучение
def train_context_optimizer(optimizer, dataset, epochs=10):
    criterion = nn.MSELoss()  # Можно использовать другие функции потерь
    opt = optim.Adam(optimizer.parameters(), lr=0.001)

    for epoch in range(epochs):
        total_loss = 0
        count = 0
        for i in range(len(dataset)):
            context, current_fragment = prepare_data(dataset[i])
            if context is None or current_fragment is None:
                continue

            compressed_context = compress_context(context, optimizer)
            predicted = predict_with_compressed_context(compressed_context, current_fragment)

            # Подготовка целевого предсказания для функции потерь
            target_input = tokenizer(current_fragment + " ", return_tensors="pt")
            with torch.no_grad():
                target_output = model(**target_input, output_hidden_states=True)
                target_embedding = target_output.hidden_states[-1].mean(dim=1)

            # Вычисление потерь
            loss = criterion(compressed_context, target_embedding)

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            count += 1


        avg_loss = total_loss / count if count > 0 else 0
        print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}")



# Запуск обучения
train_context_optimizer(optimizer, dataset)

In [1]:
import torch
from transformers import GPT2LMHeadModel
import openvino.runtime as ov
from openvino.tools.mo import convert_model
from datasets import load_dataset
from torch import nn, optim
import time

# Проверка доступности GPU от Intel
device = "GPU" if "GPU" in ov.Core().available_devices else "CPU"
print(f"Using device: {device}")

# Загрузка модели GPT-2 и перевод её в режим оценки
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.eval()

# Экспорт модели GPT-2 в ONNX формат с opset версией 14
dummy_input = torch.randint(0, 50257, (1, 10))
torch.onnx.export(model, dummy_input, "gpt2.onnx", opset_version=14)

# Конвертация ONNX модели в формат OpenVINO IR с помощью Model Optimizer
ov_model = convert_model("gpt2.onnx", input_shape=[1, 10])

# Сохранение модели в формате IR
compiled_model = ov.compile_model(ov_model, device)


# Определение модуля оптимизации контекста
class ContextOptimizer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ContextOptimizer, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, input_size)
        self.relu = nn.ReLU()

    def forward(self, context):
        x = self.relu(self.fc1(context))
        x = self.fc2(x)
        return x


# Параметры оптимизатора
input_size = 768  # Размер скрытого состояния модели GPT-2
hidden_size = 256
optimizer = ContextOptimizer(input_size, hidden_size).to("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка датасета "wikitext"
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")


# Функция для подготовки данных: извлечение контекста и текущего фрагмента
def prepare_data(example, max_length=50):
    text = example["text"].strip()
    if len(text) == 0 or "." not in text:
        return None, None

    # Разделение текста на предысторию и текущий фрагмент
    split_point = text.find(".") + 1
    if split_point >= len(text):
        return None, None

    context = text[:split_point].strip()  # Предыстория до первой точки
    current_fragment = text[split_point : split_point + max_length].strip()  # Текущий фрагмент

    if len(context) == 0 or len(current_fragment) == 0:
        return None, None

    return context, current_fragment


# Функция для сжатия контекста
def compress_context(context, optimizer):
    # Получение эмбеддингов контекста из токенов
    inputs = torch.tensor([tokenizer.encode(context)]).to("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        outputs = model(inputs, output_hidden_states=True)
        context_embedding = outputs.hidden_states[-1].mean(dim=1)  # Среднее по последнему слою

    # Сжатие контекста через оптимизатор
    compressed_context = optimizer(context_embedding)
    return compressed_context


# Функция предсказания на основе сжатого контекста с использованием OpenVINO
def predict_with_compressed_context(compressed_context, current_fragment):
    # Подготовка входных данных
    input_ids = torch.tensor([tokenizer.encode(current_fragment)]).to("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {"input_ids": input_ids.cpu().numpy()}

    # Выполнение вывода с помощью OpenVINO
    results = compiled_model(inputs)
    logits = results[compiled_model.output(0)]

    # Получение предсказаний
    predicted_index = logits.argmax(axis=-1).item()
    predicted_token = tokenizer.decode([predicted_index])
    return predicted_token


# Функция потерь и обучение с прогнозом времени
def train_context_optimizer(optimizer, dataset, epochs=10):
    criterion = nn.MSELoss()  # Можно использовать другие функции потерь
    opt = optim.Adam(optimizer.parameters(), lr=0.001)

    for epoch in range(epochs):
        total_loss = 0
        count = 0
        start_time = time.time()
        times = []

        for i in range(len(dataset)):
            step_start_time = time.time()

            context, current_fragment = prepare_data(dataset[i])
            if context is None or current_fragment is None:
                continue

            compressed_context = compress_context(context, optimizer)
            predicted = predict_with_compressed_context(compressed_context, current_fragment)

            # Подготовка целевого предсказания для функции потерь
            target_input = torch.tensor([tokenizer.encode(current_fragment + " ")]).to("cuda" if torch.cuda.is_available() else "cpu")
            with torch.no_grad():
                target_output = model(target_input, output_hidden_states=True)
                target_embedding = target_output.hidden_states[-1].mean(dim=1)

            # Вычисление потерь
            loss = criterion(compressed_context, target_embedding)

            opt.zero_grad()
            loss.backward()
            opt.step()

            total_loss += loss.item()
            count += 1

            # Оценка времени для первых 5 шагов
            step_end_time = time.time()
            times.append(step_end_time - step_start_time)

            if count == 5:
                avg_step_time = sum(times) / len(times)
                remaining_steps = len(dataset) - count
                estimated_time = avg_step_time * remaining_steps
                print(f"Estimated time for epoch {epoch+1}: {estimated_time / 60:.2f} minutes")

        avg_loss = total_loss / count if count > 0 else 0
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - start_time
        print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}, Epoch Time: {epoch_time / 60:.2f} minutes")


# Запуск обучения
train_context_optimizer(optimizer, dataset)

c:\git\forecast-py-12\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: GPU


c:\git\forecast-py-12\.venv\Lib\site-packages\transformers\modeling_attn_mask_utils.py:86: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
c:\git\forecast-py-12\.venv\Lib\site-packages\transformers\modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:


OnnxExporterError: Module onnx is not installed!

: 